In [ ]:
from langchain.document_loaders import UnstructuredPDFLoader, OnlinePDFLoader 
from langchain.text_splitter import RecursiveCharacterSplitter 

# # Load data

In [ ]:
loader = UnstructuredPDFLoader("../path/to/pdf")
# loader = OnlinePDFLoader("https://link-to-pdf")

In [ ]:
data = loader.load()

In [ ]:
print(f"You have {len(data)} document(s) in your data")
print(f"There are {len(data[0].page_content)} characters in your document")

# Chunk Your Data into smaller doc 

In [ ]:
text_splitter = RecursiveCharacterSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(data)

# Create embedding of your documents to get ready for semantic search

In [ ]:
from langchain.vectore import Chroma, Pinecone
from langchain.embeddings.openai import OpenAIEmbedding
import pinecone 

In [ ]:
#initialize pinecone 
pinecone.init(
    api_key=PINECONE_API_KEY_ENV # find at app.pincone.io
    environment=PINECONE_API_KEY_ENV # next to api_key in the console
)
index_name = "langchain2" # create in pinecone dashboard 

In [ ]:
docsearch = Pinecone.from_text([t.page_content for t in texts], embeddings, index_name=index_name)

# Query those docs to get your answer back

In [ ]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI 


In [ ]:
llm = OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)
chain = load_qa_chain(llm, chain_type="stuff")

In [ ]:
query = "What is data science"
doc = docsearch.similarity_search(query, include_metadata=True)

In [ ]:
chain.run(input_document=doc, question=query)